In [2]:
import pandas as pd
import ssl
import numpy as np
ssl._create_default_https_context = ssl._create_unverified_context
df = pd.read_csv("https://raw.githubusercontent.com/cwhitz/git_intro_to_ml/master/heart.csv")
import math

In [32]:
from sklearn.utils import shuffle
df = shuffle(df)
y = df.target
X = df.drop("target",axis=1)
Xtrain= X.head(200)
Xtest = X.tail(103)
ytrain= y.head(200)
ytest = y.tail(103)

In [102]:

class Entropy: 
    def entropy(self,node):
        entropy = 0
        for classVal in self.class_probs:
            class_prob = np.count_nonzero(node==classVal)/len(node)
            if class_prob != 0:
                entropy += -class_prob*math.log(class_prob,2)
        return entropy
    def entropy_score(self,node_left,node_right):
        m = len(node_left)+len(node_right)
        i = len(node_left)
        return i/m*self.entropy(node_left.target) + (m-i)/m*self.entropy(node_right.target)
    
    def findEntropyFromCol(self,node,col):
        vals = self.getColValues(node,col)
        splits = [(vals[i]+vals[i+1])/2 for i in range(len(vals)-1)]
        nodes = [node[col]<=split for split in splits]  
        impurities = [self.entropy_score(node[indx],node[~indx]) for indx in nodes]
        return {"entropy":min(impurities),"col":col, "split": splits[impurities.index(min(impurities))]}
  
    
    
class Node:
    def __init__(self,minEntropy):
        self.col = minEntropy["col"]
        self.split = minEntropy["split"]
        self.entropy = minEntropy["entropy"]
    
    def newLevel(self,x):
        return x[self.col]<=self.split

In [103]:
class DecisionTree(Entropy,SplitFunctions):
    def __init__(self,X,y,maxDepth):
        self.cols = X.columns
        self.node = pd.concat([X,y],axis=1)
        self.class_probs = np.unique(y)
        self.maxDepth = maxDepth
        self.Tree = self.DecisionTree(df,0)
        
    def findMajority(self,node):
        vals = node.target.value_counts()
        return vals.index[list(vals).index(max(vals))]
    
    def getColValues(self,node, col):
        return sorted(list(node[col].unique()))
    
    def findMinEntropy(self,node):
        minEntropy = {"entropy":1,"col":None, "split": None}
        
        for col in self.cols:
            
            if len(self.getColValues(node,col))<=1: continue
            newEntropy = self.findEntropyFromCol(node, col)

            if newEntropy["entropy"] <= minEntropy["entropy"]:
                minEntropy = newEntropy
                
        return minEntropy
    
    def DecisionTree(self,node, depth):
        cols_with_var_dist = np.count_nonzero([len(self.getColValues(node,col))>1 for col in node.columns if col!="target"])
        
        if depth == self.maxDepth or self.entropy(node)==0 or cols_with_var_dist==0:
            return [self.findMajority(node)]
       
        minEntropy = self.findMinEntropy(node)
        
        indx = node[minEntropy["col"]]<=minEntropy["split"]
        node_left, node_right = node[indx],node[~indx]
       
        return [Node(minEntropy),self.DecisionTree(node_left,depth+1),self.DecisionTree(node_right,depth+1)]
    
    
    

In [104]:
Tree = DecisionTree(Xtrain,ytrain,2).Tree
print(Tree)

[<__main__.Node object at 0x7ff837a45130>, [<__main__.Node object at 0x7ff837a790a0>, [1], [0]], [<__main__.Node object at 0x7ff837a79670>, [1], [1]]]


In [105]:
class Predict:
    def __init__(self,x,Tree):
        while len(Tree) > 1:
            treeLevel = Tree[0]
            direction = 1 if treeLevel.newLevel(x) else 2
            Tree = Tree[direction]
        self.pred = Tree[0]

In [107]:
false = 0
for i in range(len(Xtest)):
    if Predict(Xtest.iloc[i],Tree).pred!=ytest.iloc[i]: 
        false += 1

print(false/len(Xtest))

0.24271844660194175


In [109]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state = 0, max_depth = 2)
clf.fit(Xtrain, ytrain)
print(clf.score(Xtest, ytest))
print(clf.tree_.max_depth)
print(np.count_nonzero(clf.predict(Xtest)!=ytest)/len(Xtest))
clf.predict(Xtest)

0.7572815533980582
2
0.24271844660194175


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1,
       1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1])

In [4]:
from sklearn import tree
from subprocess import check_call

In [87]:
text_representation = tree.export_text(clf)
print(text_representation)

|--- feature_2 <= 0.50
|   |--- feature_11 <= 0.50
|   |   |--- class: 1
|   |--- feature_11 >  0.50
|   |   |--- class: 0
|--- feature_2 >  0.50
|   |--- feature_0 <= 56.50
|   |   |--- class: 1
|   |--- feature_0 >  56.50
|   |   |--- class: 1



In [158]:
math.log(.5,2)

-1.0

In [ ]:
{0.25(g1(xn) + g2(xn))2 - 2yn(g1(xn) + g2(xn))+yn2}